In [1]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

np.random.seed(0)

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = float(tmp_val) - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [2]:
class LogicGate:
    def __init__(self, x_data, t_data, learning_rate, iteration_count):
        
        # 가중치 W 형상을 자동으로 구하기 위해 입력데이터가 vector 인지,
        # 아니면 matrix 인지 체크 후, 
        # self.xdata 는 무조건 matrix 로 만들어 주면 코드 일관성이 있음
        if x_data.ndim == 1:    # vector
            self.x_data = x_data.reshape(len(x_data), 1)
            self.t_data = t_data.reshape(len(t_data), 1)
        elif x_data.ndim == 2:  # matrix
            self.x_data = x_data
            self.t_data = t_data
        
        self.learning_rate = learning_rate
        self.iteration_count = iteration_count
        
        self.W = np.random.rand(self.x_data.shape[1], 1)
        self.b = np.random.rand(1)
        
        self.loss_val_list = []
        
        print("ClassificationTest Object is created.\n")
        
    def get_W_b(self):
        return self.W, self.b
        
    def __loss_func(self):
        delta = 1e-7
        
        z = np.dot(self.x_data, self.W) + self.b
        y = sigmoid(z)
    
        return -np.sum((self.t_data*np.log10(y + delta)) + ((1 - self.t_data)*np.log10((1 - y) + delta)))
    
    # 손실함수 값 계산 함수
    # 입력변수 x, t : numpy type
    def __loss_val(self):
        delta = 1e-7
        
        z = np.dot(self.x_data, self.W) + self.b
        y = sigmoid(z)
    
        return -np.sum((self.t_data*np.log10(y + delta)) + ((1 - self.t_data)*np.log10((1 - y) + delta)))

    def train(self):
        f = lambda x : self.__loss_func()
        
        print("Initial error value = ", self.__loss_val() , "\n", "Initial W = ", self.W, "\n", ", b = ", self.b)

        start_time = datetime.now()

        for step in range(self.iteration_count):
            self.W -= self.learning_rate * numerical_derivative(f, self.W)
            self.b -= self.learning_rate * numerical_derivative(f, self.b)
            
            if step % (int)(0.05*self.iteration_count) == 0:
                print("step = ", step, "loss value = ", self.__loss_val())
                self.loss_val_list.append(self.__loss_val())
        
        end_time = datetime.now()

        print("")
        print("Elapsed Time => ", end_time - start_time)
        
    def display_lossval_trend(self):
        plt.title('Loss Value Trend')
        plt.xlabel('epochs ( X 1000)')
        plt.ylabel('loss value')
        plt.grid()

        plt.plot(self.loss_val_list, ls='--', lw=2, label='lr={}, epoch={}'.format(self.learning_rate, self.iteration_count))
        plt.legend(loc='best')

        plt.show()
    
    # 학습을 마친 후, 임의의 데이터에 대해 미래 값 예측 함수
    # 입력변수 x : numpy type
    def predict(self, test_data):
        z = np.dot(test_data, self.W) + self.b
        y = sigmoid(z)
        
        if y >= 0.5:
            result = 1
        else:
            result = 0
    
        return y, result
    
     # 정확도 예측 함수
    def accuracy(self, test_xdata, test_tdata):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(xdata)):
            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_val = len(matched_list) / len(test_xdata)
        
        return accuracy_val

In [7]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

NAND_obj = LogicGate(xdata, tdata, 1e-2, 20001)

NAND_obj.train()

NAND_obj.accuracy(xdata, tdata)

ClassificationTest Object is created.

Initial error value =  4.192086064664454 
 Initial W =  [[0.43758721]
 [0.891773  ]] 
 , b =  [0.96366276]
step =  0 loss value =  4.1858485268221
step =  1000 loss value =  3.9085922124603516
step =  2000 loss value =  3.9075205787958445
step =  3000 loss value =  3.9075036947586685
step =  4000 loss value =  3.9075034194499603
step =  5000 loss value =  3.9075034149442427
step =  6000 loss value =  3.907503414870474
step =  7000 loss value =  3.9075034148692662
step =  8000 loss value =  3.9075034148692467
step =  9000 loss value =  3.907503414869246
step =  10000 loss value =  3.9075034148692467
step =  11000 loss value =  3.9075034148692462
step =  12000 loss value =  3.9075034148692467
step =  13000 loss value =  3.9075034148692467
step =  14000 loss value =  3.9075034148692467
step =  15000 loss value =  3.9075034148692467
step =  16000 loss value =  3.9075034148692467
step =  17000 loss value =  3.9075034148692467
step =  18000 loss value =

0.75

In [6]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

OR_obj = LogicGate(xdata, tdata, 1e-2, 20001)

OR_obj.train() 

ClassificationTest Object is created.

Initial error value =  4.192086064664454 
 Initial W =  [[0.43758721]
 [0.891773  ]] 
 , b =  [0.96366276]
step =  0 loss value =  4.1858485268221

Elapsed Time =>  0:00:03.719640


In [6]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])

AND_obj = LogicGate(xdata, tdata, 1e-2, 20001)

AND_obj.train()

ClassificationTest Object is created.

Initial error value =  7.910928813853937 
 Initial W =  [[0.54488318]
 [0.4236548 ]] 
 , b =  [0.64589411]
step =  0 loss value =  7.720901617844199
step =  1000 loss value =  3.908035429063945
step =  2000 loss value =  3.9075120026003116
step =  3000 loss value =  3.907503555235623
step =  4000 loss value =  3.907503417167037
step =  5000 loss value =  3.9075034149068664
step =  6000 loss value =  3.9075034148698626
step =  7000 loss value =  3.9075034148692565
step =  8000 loss value =  3.9075034148692467
step =  9000 loss value =  3.9075034148692462
step =  10000 loss value =  3.9075034148692467
step =  11000 loss value =  3.9075034148692467
step =  12000 loss value =  3.9075034148692467
step =  13000 loss value =  3.9075034148692467
step =  14000 loss value =  3.9075034148692467
step =  15000 loss value =  3.9075034148692467
step =  16000 loss value =  3.9075034148692467
step =  17000 loss value =  3.9075034148692467
step =  18000 loss value 